In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei'] 
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

train_df = pd.read_csv("train.csv", low_memory = False)
test_df = pd.read_csv("test.csv", low_memory = False)
# train_df

# prices = pd.DataFrame({"price":train_df["SalePrice"], "log(price + 1)":np.log1p(train_df["SalePrice"])})
# plt.ylabel(u"數量")
# plt.title(u"房屋總價")
# prices.hist()
# plt.show()

# np.log1p --> log(1+x)
y_train = np.log1p(train_df.pop("SalePrice"))
all_df = pd.concat((train_df, test_df), axis=0)



In [2]:
# train_df.loc[:,numric_cols]

In [3]:
# 2.特徵處理

all_df['MSSubClass'] = all_df['MSSubClass'].astype(str)
# print(all_df['MSSubClass'].value_counts())
# print(pd.get_dummies(all_df['MSSubClass'],prefix='MSSubClass').head(5))


#將所有category資料進行OneHot
all_dummy_df = pd.get_dummies(all_df)
# all_dummy_df.to_csv('G:/KNNtest/HousePrice/One_hot.csv')      #將所有Category資料進行OneHot之後儲存

# print(all_dummy_df.isnull().sum().sort_values(ascending=False).head(20))      #讀取出所有特徵中缺失項的數目
#使用平均值來填補空缺
mean_cols = all_dummy_df.mean()
all_dummy_df = all_dummy_df.fillna(mean_cols)

#標準化numeric資料，一般regression分類器需要將源資料放在一個標準分佈內（x-x'）/s
numric_cols = all_df.columns[all_df.dtypes != 'object']     #找出numeric資料
numric_cols_means = all_dummy_df.loc[:,numric_cols].mean()
numric_cols_std = all_dummy_df.loc[:,numric_cols].std()      #標準差s
all_dummy_df.loc[:,numric_cols] = (all_dummy_df.loc[:,numric_cols] - numric_cols_means)/numric_cols_std

In [4]:
# model
dummy_train_df = all_dummy_df.loc[train_df.index]
dummy_test_df = all_dummy_df.loc[test_df.index]

X_train = dummy_train_df.values
X_test = dummy_test_df.values

In [5]:
'''
(1)ridge模型測試
'''
alphas = np.logspace(-3, 2, 50) #-3,2表示開始點和結束點10的冪次方，50代表元素個數
test_scores = []
for alpha in alphas:
    clf = Ridge(alpha)
    test_score = np.sqrt(-cross_val_score(clf, X_train, y_train, cv=10, scoring='neg_mean_squared_error'))
    test_scores.append(np.mean(test_score))
plt.plot(alphas,test_scores)
plt.title('Aplha VS CV Error')
plt.show()                                 #由圖知道在alpha=15左右交叉驗證誤差基本最小,約為0.135

ValueError: Found input variables with inconsistent numbers of samples: [2919, 1460]